In [22]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.16.0"]])

(ns chp2
    (:refer-clojure :exclude [partial zero? + - * / ref compare])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]
              [sicmutils.calculus.manifold :refer [Rn, R2, make-patch]]))
(def render (comp ->infix simplify))

#'chp2/render

In [23]:
(clojure.repl/source R2)

(def R2 (make-manifold Rn 2))


nil

## 3.1 Coordinate Functions

In [5]:
(clojure.repl/source R2-rect)

(def R2-rect (coordinate-system-at :rectangular :origin R2))


nil

In [6]:
(clojure.repl/source R2-polar)

(def R2-polar (coordinate-system-at :polar-cylindrical :origin R2))


nil

In [8]:
(def R2-rect-chi (chart R2-rect))
(def R2-rect-chi-inverse (point R2-rect))
(def R2-polar-chi (chart R2-polar))
(def R2-polar-chi-inverse (point R2-polar))

#'chp2/R2-polar-chi-inverse

In [10]:
(render ((compose R2-polar-chi R2-rect-chi-inverse) (up 'x0 'y0)))

"up(sqrt(x0² + y0²), atan(y0, x0))"

In [11]:
(render ((compose R2-rect-chi R2-polar-chi-inverse) (up 'r0 'theta0)))

"up(r0 cos(theta0), r0 sin(theta0))"

In [14]:
 ((D (compose R2-rect-chi R2-polar-chi-inverse)) (up 'r0 'theta0))

(down (up (cos theta0) (sin theta0)) (up (* r0 (- (sin theta0))) (* r0 (cos theta0))))

In [15]:
(render ((D (compose R2-rect-chi R2-polar-chi-inverse)) (up 'r0 'theta0)))

"down(up(cos(theta0), sin(theta0)), up(- r0 sin(theta0), r0 cos(theta0)))"

## 2.2 Manifold Functions
### Manifold Functions Are Coordinate Independent

In [19]:
;; a signiture for functions that map an up structure of two reals to a real.
(def R2->R '(-> (UP Real Real) Real))

#'chp2/R2->R

In [21]:
;; define the manifold function
(def f 
    (compose (literal-function 'f-rect R2->R) R2-rect-chi))

#'chp2/f

In [22]:
;; define a manifold point using its rectangular coordinates
(def R2-rect-point (R2-rect-chi-inverse (up 'x0 'y0)))

#'chp2/R2-rect-point

In [23]:
;; describe the same point using its polar coordinates
(def cooresponding-polar-point
    (R2-polar-chi-inverse
     (up (sqrt (+ (square 'x0) (square 'y0)))
         (atan 'y0 'x0))))

#'chp2/cooresponding-polar-point

In [24]:
(f R2-rect-point)

(f-rect (up x0 y0))

In [25]:
(f cooresponding-polar-point)

(f-rect (up x0 y0))

### Naming Coordinate Functions

These coordinate functions allow us to extrace coordinates from a point, independent of the coordinate system used to specify the point. 

In [37]:
(let-coordinates
 [[x y] R2-rect]
 (x (R2-rect-chi-inverse (up 'x0 'y0))))

x0

In [38]:
(let-coordinates
 [[x y] R2-rect]
 (y (R2-rect-chi-inverse (up 'x0 'y0))))

y0

In [40]:
(let-coordinates
 [[r theta] R2-polar]
 (r (R2-rect-chi-inverse (up 'x0 'y0))))

(sqrt (+ (expt x0 2) (expt y0 2)))

In [41]:
(let-coordinates
 [[r theta] R2-polar]
 (theta (R2-rect-chi-inverse (up 'x0 'y0))))

(atan y0 x0)

With an abuse of notation, we define new manifold functions in terms of the coordinate functions. and expression like `cos(r)` can either mean the cosine of the angle `r` if `r` is a number or the composition `(compose cos r)` if `r` is a function

In [43]:
(let-coordinates
 [[x y] R2-rect
  [r theta] R2-polar]
 (def h (+ (* x (square r)) (cube y)))
 (h R2-rect-point))

(+ (* x0 (+ (expt x0 2) (expt y0 2))) (expt y0 3))

In [44]:
(let-coordinates
 [[x y] R2-rect
  [r theta] R2-polar]
 (def h (+ (* x (square r)) (cube y)))
 (h (R2-polar-chi-inverse (up 'r0 'theta0))))

(+ (* r0 (cos theta0) (expt r0 2)) (expt (* r0 (sin theta0)) 3))

In [46]:
(clojure.repl/source let-coordinates)

(defmacro let-coordinates [& args]
  `(cc/let-coordinates ~@args))


nil

### Ex 2.1 Curves

In [47]:
(let-coordinates
 [[r theta] R2-polar]
 ((- r (* 2 'a (+ 1 (cos theta))))
  ((point R2-rect) (up 'x 'y))))

(- (sqrt (+ (expt x 2) (expt y 2))) (* 2 a (+ 1 (cos (atan y x)))))

In [48]:
(let-coordinates
 [[r theta] R2-polar]
 (render ((- r (* 2 'a (+ 1 (cos theta))))
  ((point R2-rect) (up 'x 'y)))))

"(-2 a x -2 a sqrt(x² + y²) + x² + y²) / sqrt(x² + y²)"

__a.__ The rectangular coordinate equation for the Lemniscate of Bernoulli is 
$$ (x^2 + y^2)^2 = 2 a^2 (x^2 - y^2) $$
FInd the expression in polar coordinates

In [54]:
(let-coordinates
 [[x y] R2-rect]
 ((- (* (+ (* x x) (* y y)) (+ (* x x) (* y y)))
     (* 2 (* 'a 'a) (- (* x x) (* y y))))
  ((point R2-polar) (up 'r 'theta))))

(- (* (+ (* r (cos theta) r (cos theta)) (* r (sin theta) r (sin theta))) (+ (* r (cos theta) r (cos theta)) (* r (sin theta) r (sin theta)))) (* 2 a a (- (* r (cos theta) r (cos theta)) (* r (sin theta) r (sin theta)))))

In [55]:
(let-coordinates
 [[x y] R2-rect]
 (render ((- (* (+ (* x x) (* y y)) (+ (* x x) (* y y)))
     (* 2 (* 'a 'a) (- (* x x) (* y y))))
  ((point R2-polar) (up 'r 'theta)))))

"-4 a² r² cos²(theta) + 2 a² r² + r⁴"

This result can be further simplified
$$-4 a^2 r^2 \cos^2 (\theta) + 2 a^2 r^2 + r^4 = r^2 - 2 a^2 \cos (2\theta) = 0$$

In [59]:
R3-rect

#object[sicmutils.calculus.manifold$$reify__20397 0xb133bbe "sicmutils.calculus.manifold$$reify__20397@b133bbe"]

__b.__ Describe a helix space curve in both rectangular and cylindrical coordinates.

A helix space curve in rectangular coordinates is
$$
\begin{align}
x(t) &= \cos(t) \\
y(t) &= \sin(t) \\
z(t) &= t
\end{align}
$$

A helix space curve in cylindrical coordinates is 
$$
\begin{align}
r(t) &= 1 \\
\theta(t) &= t \\
z(t) &= t
\end{align}
$$

To derive the rectangular coordinates from the cylindrical coordinates, we can express the cylindrical coordinates in two equations
$$
\begin{align}
r - 1 &= 0 \\
\theta - z &= 0 \\
\end{align}
$$
Expressing these two residuals in rectangular coordinates gives


In [10]:
(let-coordinates
 [[r theta z] R3-cyl]
 ((- theta z)
  ((point R3-rect) (up 'x 'y 'z))))

(- (atan y x) z)

In [64]:
(let-coordinates
 [[r theta z] R3-cyl]
 ((- r 1)
  ((point R3-rect) (up 'x 'y 'z))))

(- (sqrt (+ (expt x 2) (expt y 2))) 1)

which are 
$$
\begin{align}
\tan (z)  &= y/x \\
\sqrt{x^2 + y^2} &= 1 \\
\end{align}
$$.
We can recover the rectangular form from these two equations.

To derive the cylindrical coordinates form from the rectangular coordinate form, we use the equations
$$
\begin{align}
x - \cos z &= 0 \\
y - \sin z &= 0 \\
\end{align}
$$
Expression these two residuals in cylindrical equation gives

In [65]:
(let-coordinates
 [[x y z] R3-rect]
 ((- x (cos z))
  ((point R3-cyl) (up 'r 'theta 'z))))

(- (* r (cos theta)) (cos z))

In [66]:
(let-coordinates
 [[x y z] R3-rect]
 ((- y (sin z))
  ((point R3-cyl) (up 'r 'theta 'z))))

(- (* r (sin theta)) (sin z))

$$
\begin{align}
r \cos (\theta)  &= \cos(z) \\
r \sin (\theta) &= \sin (z) \\
\end{align}
$$.

### Ex 2.2 Stereographic Projection

In [11]:
S2-Riemann

#object[sicmutils.calculus.manifold$__GT_Stereographic$ctor$reify__20810 0x34e42893 "sicmutils.calculus.manifold$__GT_Stereographic$ctor$reify__20810@34e42893"]

In [12]:
S2-spherical

#object[sicmutils.calculus.manifold$__GT_S2_coordinates$ctor$reify__20770 0xe9c7019 "sicmutils.calculus.manifold$__GT_S2_coordinates$ctor$reify__20770@e9c7019"]

The following procedure gives the colatitude and longitude of a point on the sphere corresponding to a point on the plane. 

In [13]:
((compose
  (chart S2-spherical)
  (point S2-Riemann)
  (chart R2-rect)
  (point R2-polar))
 (up 'rho 'theta))

(up (acos (/ (+ (expt rho 2) -1) (+ (expt rho 2) 1))) theta)

In [16]:
(render *1)

"up(acos((rho² + -1) / (rho² + 1)), theta)"

__Perform an analogous computation to get the polar coordinates of the point on the plane corresponding to a point on the sphere given by its colatitude and longitude__

In [17]:
((compose
  (chart R2-polar)
  (point R2-rect)
  (chart S2-Riemann)
  (point S2-spherical))
 (up 'colat 'long))

(up (sqrt (/ (+ (* -1 (cos colat)) -1N) (+ (cos colat) -1N))) long)

In [18]:
(render *1)

"up(sqrt((- cos(colat) + -1) / (cos(colat) + -1)), long)"